In [19]:
from tqdm import tqdm
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("blue_score.ipynb"), "..")))
from transformers_models.marian.marianMT import MarianMt

In [20]:
def retrieve_data(filenames):

    english_cleaned = []
    german_cleaned = []
    base_path = os.path.join(os.path.dirname("blue_score.ipynb"), "..", "data")
    en_path = os.path.join(base_path, filenames[0])
    de_path = os.path.join(base_path, filenames[1])

    with open(en_path, mode = "r", encoding = "utf-8") as f_en, \
        open(de_path, mode = "r", encoding = "utf-8") as f_de:
        for en, de in tqdm(zip(f_en.readlines(), f_de.readlines())):
            english_cleaned.append(en[:-1])
            german_cleaned.append(de[:-1])

    return (english_cleaned, german_cleaned)


In [21]:
def translated(n, model):
    return model.translate_text(n)

In [24]:
cleaned = retrieve_data(["test.en", "test.de"])
english_cleaned_to_translate = cleaned[0]
german_cleaned_reference = cleaned[1]

model = MarianMt("Helsinki-NLP/opus-mt-en-de")
translations = model.batch_translate(english_cleaned_to_translate)

print(translations)


3002it [00:00, 1002731.59it/s]


AttributeError: 'MarianMt' object has no attribute 'batch_translate'

In [ ]:
import nltk
nltk.download('punkt')  
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
list_of_references = []
hypotheses = []

for ref, trans in zip(german_cleaned_reference, translations):

    ref_tokens = nltk.word_tokenize(ref.lower())
    trans_tokens = nltk.word_tokenize(trans.lower())

    list_of_references.append([ref_tokens])
    
    hypotheses.append(trans_tokens)


In [ ]:
score = round(corpus_bleu(list_of_references, hypotheses), 3)
print("Corpus BLEU:", score)  
print("Corpus BLEU (%):", score * 100)
